In [ ]:
import os
import urllib.request
from datetime import timedelta, date
from osgeo import gdal, ogr, osr
import subprocess

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield end_date - timedelta(n)

def process_grib_file(input_file, base_output_name):
    # open the dataset and get the geo transform matrix
    ds = gdal.Open(input_file)
    gt = ds.GetGeoTransform()
    srs = osr.SpatialReference()
    srs.ImportFromWkt(ds.GetProjection())

    # create a new image with the data we want
    driver = gdal.GetDriverByName('GTiff')
    temp_file = f'data/tiffs/{base_output_name}_temp.tiff'
    outRaster = driver.Create(temp_file, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform(gt)
    outRaster.SetProjection(ds.GetProjection())
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(ds.GetRasterBand(1).ReadAsArray())

    # Set the nodata value
    outband.SetNoDataValue(9999)
    outband.FlushCache()
    outRaster = None  # ensure the data gets written

    # Delete the .grib2 file
    os.remove(input_file)

start_date = date(2012, 1, 1)
end_date = date(2023, 5, 22)  # Adjust this to the current date if needed

directory = "data/pm25"
if not os.path.exists(directory):
    os.makedirs(directory)

for single_date in daterange(start_date, end_date):
    date_str = single_date.strftime("%Y%m%d")
    file_path = os.path.join(directory, f'{date_str}_current_pm25.grib2')
    base_output_name = f'{date_str}_current_pm25'
    output_file = os.path.join("data/tiffs", f"{base_output_name}_temp.tiff")
    
    if os.path.isfile(output_file):
        print(f"Output file for date {date_str} already exists, skipping.")
        continue

    url = f'https://s3-us-west-1.amazonaws.com//files.airnowtech.org/airnow/{single_date.year}/{date_str}/current_pm25.grib2'

    try:
        urllib.request.urlretrieve(url, file_path)
        process_grib_file(file_path, base_output_name)
    except urllib.error.URLError:
        print(f"Could not download file for date {date_str}")


In [2]:
import rasterio
import geopandas as gpd
from h3 import h3
from shapely.geometry import Polygon
import numpy as np
import rasterio.features
import rasterio

def raster_to_polygons(tiff_path):
    """
    Convert a GeoTIFF into a GeoDataFrame of polygons
    """
    with rasterio.open(tiff_path) as src:
        image = src.read(1)  # assumes a single band
        results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(rasterio.features.shapes(image, transform=src.transform)) 
        )
        geoms = list(results)
        return gpd.GeoDataFrame.from_features(geoms)

def polygons_to_h3(gdf, resolution=6):
    """
    Convert polygons in a GeoDataFrame to H3 hexagons of a specific resolution
    """
    def polyfill(geom):
        return h3.polyfill(geom.__geo_interface__, resolution)

    gdf['hexagons'] = gdf['geometry'].apply(polyfill)
    return gdf




In [3]:
tiff_path = '/data/tiffs/20221030_current_pm25_temp.tiff'
gdf = raster_to_polygons(tiff_path)
hex_gdf = polygons_to_h3(gdf)

In [6]:
hex_gdf

,geometry,raster_val,hexagons
0,"POLYGON ((-111.20286 59.40841, -111.15785 59.4...",12.0,{}
1,"POLYGON ((-111.96807 58.71052, -111.94556 58.7...",13.0,{}
2,"POLYGON ((-112.01308 58.55293, -112.01308 58.4...",14.0,{}
3,"POLYGON ((-112.03558 58.44037, -112.03558 58.3...",15.0,{}
4,"POLYGON ((-112.03558 58.37283, -112.03558 58.3...",16.0,{}
...,...,...,...
188820,"POLYGON ((-101.16515 20.01126, -101.16515 19.9...",85.0,{}
188821,"POLYGON ((-101.09763 20.01126, -101.09763 19.9...",88.0,{}
188822,"POLYGON ((-101.07512 20.01126, -101.07512 19.9...",89.0,{}
188823,"POLYGON ((-100.42245 20.75417, -100.39994 20.7...",93.0,{}


In [4]:
import h3
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, Point

# Define the bounding box of CONUS
min_lat, max_lat = 24.396308, 49.384358
min_lon, max_lon = -125.000000, -66.934570

# Approximate distance (in degrees) between hexagons at resolution 6
# The edge length for resolution 6 is approximately 4.12 km.
delta_lat = 4.12 / 111  # Approximate conversion factor from km to degrees latitude
delta_lon = 4.12 / 85   # Approximate conversion factor from km to degrees longitude (this varies with latitude)

# Create arrays for latitudes and longitudes
lats = np.arange(min_lat, max_lat, delta_lat)
lons = np.arange(min_lon, max_lon, delta_lon)

# Initialize an empty list to store the rows
rows = []

# Fill the area with H3 hexagons
for lat in lats:
    for lon in lons:
        # Get the H3 index of the point
        h3_index = h3.geo_to_h3(lat, lon, 6)
        
        # Convert the H3 index to a polygon
        poly = h3.h3_to_geo_boundary(h3_index, geo_json=True)
        poly = Polygon(poly)
        
        # Calculate the centroid of the polygon
        centroid = poly.centroid
        
        # Add the polygon and its centroid to the list
        rows.append({'geometry': centroid, 'h3_index': h3_index})

# Convert the list of rows to a GeoDataFrame
gdf = gpd.GeoDataFrame(rows)

# Set the CRS of the GeoDataFrame to WGS84 (EPSG:4326)
gdf.crs = "EPSG:4326"

# Save the H3 grid to a shapefile
gdf.to_file("conus_h3_grid.shp")


### Raster to H3 values

In [ ]:
import re
import rasterio
import rasterio.mask
from shapely.geometry import mapping

# Load the H3 grid as a GeoDataFrame
gdf = gpd.read_file("data/h3/conus_h3_grid.shp")

# Define the input file name
input_file_name = '20180816_current_pm25.tiff'

# Extract the date from the input file name
date_match = re.search(r'\d{8}', input_file_name)
if date_match:
    date_str = date_match.group()
else:
    raise ValueError(f"No date found in file name: {input_file_name}")

# Load the GeoTIFF as a raster
with rasterio.open(input_file_name) as src:
    # For each hexagon in the H3 grid
    for index, row in gdf.iterrows():
        # Convert the hexagon's geometry to the format required by rasterio
        geometry = [mapping(row.geometry)]

        # Mask the raster with the hexagon
        out_image, out_transform = rasterio.mask.mask(src, geometry, crop=True, filled=False)
        out_meta = src.meta.copy()

        # The result is a 2D numpy array with the raster values within the hexagon
        # We need to ignore the nodata values (default is -inf)
        values = out_image[out_image != out_meta['nodata']]

        # Now you can calculate the statistics of the raster cells within the hexagon
        # For example, let's calculate the mean
        mean = values.mean()

        # Add the mean to the GeoDataFrame, now under the 'pm2.5' column
        gdf.loc[index, 'pm2.5'] = mean

        # Remove rows with NaN 'pm2.5' values
        gdf = gdf.dropna(subset=['pm2.5'])

# Save the updated GeoDataFrame to a new shapefile, including the date in the file name
gdf.to_file(f"conus_h3_grid_with_pm2.5_{date_str}.shp")


In [ ]:
gdf.head(5)

,h3_index,geometry,pm2.5
0,8650852dfffffff,POINT (-124.98863 24.41565),NaN
1,8650852d7ffffff,POINT (-124.94276 24.37168),NaN
2,865081927ffffff,POINT (-124.87963 24.38698),NaN
3,865081927ffffff,POINT (-124.87963 24.38698),NaN
4,865081937ffffff,POINT (-124.81644 24.40226),NaN
